### Micrograd in PyTorch
- This project is inspired by Andrej Karpathy's Micrograd.
- Micrograd is a simplified Autograd system.
- This project is intended to help me learn and understand the implementation of simplified autograd.
- My code can be a bit retarded tho....

In [1]:
import torch 
import torch.autograd as autograd

In [14]:
class Value:
    """Stores a single scalar value and its gradient"""
    def __init__(self, data, requires_grad=True):
        if isinstance(data, torch.Tensor):
            self.data = data.clone().detach().requires_grad_(requires_grad)
        else:
            self.data = torch.tensor(data, requires_grad=requires_grad)

    def __add__(self, other):
        if isinstance(other, Value):
            return Value(self.data + other.data)
        else:
            return Value(self.data + other)

    def __mul__(self, other):
        if isinstance(other, Value):
            return Value(self.data * other.data)
        else:
            return Value(self.data * other)

    def __pow__(self, other):
        return Value(self.data ** other)

    def relu(self):
        return Value(torch.relu(self.data))

    def backward(self):
        self.data.backward()

    def __neg__(self):
        return Value(-self.data)

    def __radd__(self, other):
        return self + other 

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return Value(other) + (-self)

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * (other ** -1)

    def __rtruediv__(self, other):
        return Value(other) * (self ** -1)

    def __repr__(self):
        return f"Value(data={self.data.item()}, grad={self.data.grad})"

### Neural Network

In [15]:
import torch 
import torch.nn as nn 

In [16]:
class Module(nn.Module):
    def __init__(self):
        super(Module, self).__init__()

    def zero_grad(self):
        for p in self.parameters():
            p.grad = None 

    def parameters(self):
        return list(self.parameters())



In [19]:
class Neuron(Module):
    def __init__(self, nin, nonlin=True):
        super(Neuron, self).__init__()
        self.w = nn.Parameter(torch.randn(nin))
        self.b = nn.Parameter(torch.zeros(1))
        self.nonlin = nonlin 

    def forward(self, X):
        act = torch.sum(self.w * x, dim=0) + self.b # Activation 
        if self.nonlin:
            return torch.relu(act)
        else:
            return act 

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

In [20]:
class Layer(Module):
    def __init__(self, nin, nout, **kwargs):
        super(Layer, self).__init__()
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def forward(self, X):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out 

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

In [23]:
class MLP(Module):
    def __init__(self, nin, nouts):
        super(MLP, self).__init__()
        sz = [nin] + nouts 
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x 

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

In [24]:
mlp = MLP(784, [256, 10])
print(mlp)

MLP of [Layer of [ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(784), ReLUNeuron(78

#### The above output is kinda gibberish but...
- 'MLP of [...]' This indicates that the network is multi-layer-perceptron
- 'LinearNeuron(256)' This indicates that the second layer has 256 'Layer of [...]]'
- 'ReLUNeuron(784)': This indicates that the first layer has 784 neurons and uses the ReLU activation of function 